# Analysis of Cosine-Similarity Model

In this notebook, a parsimonious version of K-Nearest Neighbors (dubbed the Cosine-Similarity Model) is proposed that results in a slightly higher accuracy than standard K-Nearest Neighbor models, along with an almost 2 times (or greater) classification speedup.  The models's speed and accuracy is a reminder that building in-house models is often the best option when solving specific problems, as opposed to using pre-built models.

## Introduction

#### Standard K-Nearest Neighbors
K-Nearest Neighbor (K-NN) algorithms are simple supervised classification algorithms that have the capability of making accurate predictions on complex problems.  K-NN is a lazy learner, so it requires no training and can thus get right to classification, making it a much faster algorithm than other classification models such as SVM, regression, multi-layer perceptron, etc..  K-NN is also non-parametric, so it makes no assumptions about the data.  Because the algorithm requires no training, data can be added or taken away seamlessly, without making any major adjustments.

Given a point $p$ to classify, a K-NN model will "compare" the passed point with each point $x_i$ the model has available to it, using some distance metric (most commonly Euclidean distance).  This process will generate the unordered set $D$ that holds the distances (similarities) between $p$ and every other point in the dataset, $x_i$, giving $d_i$.  Next, the algorithm pulls the $k$ lowest distances (greatest similarities) from $D$, and uses some voting technique to classify $p$ as being a member of some class $C$.  See [this](https://importq.wordpress.com/2017/11/24/mnist-analysis-using-knn/) tutorial for a more thorough description of K-NN.

#### The Cosine-Similarity Model
The Cosine-Similarity Model works in the same general way as most K-NN classifiers.  The primary difference with the Cosine-Similarity Model is in its name: it uses cosine-similarity as a distance metric instead of the more common Euclidean or Manhatten distance.  Cosine-similarity is given by

$$
similarity=cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{||\vec{a}|| ||\vec{b}||}
$$

where $\vec{a}$ and $\vec{b}$ are vectors whose similarity is returned.  Cosine similarity is often not a perfect distance metric, as it doesn't work on negative data, and violates the triangle inequality.  However for certain problems (as shown below) it is a solid choice.

After testing the Cosine-Similarity Model on the MNIST data set, it is found that the classifier is both faster and just as, if not more accurate than go-to K-NN models from the Scikit-Learn library.  In the analysis below, I will build out the Cosine-Similarity Model, and run it on the MNIST data set.  I will then test a go-to K-NN model from Scikit-Learn on the MNIST dataset, finally comparing both the accuracy and classification time of the two models in a variety of situations.  All tests were run on a Intel Core 3570K CPU.

## Analysis

Start with required imports

In [1]:
import numpy as np
import heapq
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import datasets, model_selection

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix  

mnist = datasets.fetch_mldata('MNIST original')
data, target = mnist.data, mnist.target

In [2]:
data.shape, target.shape

((70000, 784), (70000,))

Below, different data sets are built out of the full MNIST data set for the purpose of testing the Cosine-Similarity Model and Scikit-Learn K-NN under different situations.

In [3]:
# make an array of indices to use for making the data sets, in random order, the same length of the MNIST dataset
indx = np.random.choice(len(target), 70000, replace=False)

# method for building datasets to test with
def mk_dataset(size):
    """makes a dataset of size "size", and returns that datasets images and targets
    This is used to make the dataset that will be stored by a model and used in 
    experimenting with different stored dataset sizes
    """
    train_img = [data[i] for i in indx[:size]]
    train_img = np.array(train_img)
    train_target = [target[i] for i in indx[:size]]
    train_target = np.array(train_target)
    
    return train_img, train_target

################################################
# TEST SETS

# will be keeping test set the same for different stored data sets
# test_data/test_target: the smaller dataset used to test model accuracy for the data sets
test_img = [data[i] for i in indx[60000:70000]]
test_img = np.array(test_img)
test_target = [target[i] for i in indx[60000:70000]]
test_target = np.array(test_target)

# Single random test image, this is just used for testing the speed at which each model can 
# classify a single point
t_img = test_img[563]
t_img = np.array([t_img])
t_target = test_target[563]
t_target = np.array([t_target])
t_target

test_img.shape, test_target.shape, t_target

((10000, 784), (10000,), array([3.]))

In [4]:
# make different datasets
sixty_x, sixty_y = mk_dataset(60000)
sixty_x.shape, sixty_y.shape

((60000, 784), (60000,))

In [5]:
fifty_x, fifty_y = mk_dataset(50000)
fifty_x.shape, fifty_y.shape

((50000, 784), (50000,))

In [6]:
fourty_x, fourty_y = mk_dataset(40000)
fourty_x.shape, fourty_y.shape

((40000, 784), (40000,))

In [7]:
thirty_x, thirty_y = mk_dataset(30000)
thirty_x.shape, thirty_y.shape

((30000, 784), (30000,))

In [8]:
twenty_x, twenty_y = mk_dataset(20000)
twenty_x.shape, twenty_y.shape

((20000, 784), (20000,))

In [9]:
ten_x, ten_y = mk_dataset(10000)
ten_x.shape, ten_y.shape

((10000, 784), (10000,))

In [10]:
one_x, one_y = mk_dataset(1000)
one_x.shape, one_y.shape

((1000, 784), (1000,))

Now we have 7 different data sets for the classifier to use, ranging from size 1,000 to size 60,000.  We also have a testing data set of size 10,000 to calculate accuracy and speed of the classifiers, as well as a smaller testing dataset of size 1 used for computing sinlge point classification speed.  

Now we build the Cosine-Similarity Classifier.  The method only takes the `test_target` argument to calculate prediction accuracy, it is not actually needed for classification.  

In [11]:
def cos_knn(k, test_data, test_target, stored_data, stored_target):
    """k: number of neighbors to use for voting
    test_data: a set of unobserved images to classify
    test_target: the labels for the test_data (for calculating accuracy)
    stored_data: the images already observed and available to the model
    stored_target: labels for stored_data
    """
    
    # find similarity for every point in test_data between every other point in stored_data
    cosim = cosine_similarity(test_data, stored_data)
    
    # get indices of images in stored_data that are most similar to any given test_data point
    top = [(heapq.nlargest((k+1), range(len(i)), i.take)) for i in cosim]
    # convert indices to numbers
    top = [[stored_target[j] for j in i[:k]] for i in top]
    
    # vote, and return prediction for every image in test_data
    pred = [max(set(i), key=i.count) for i in top]
    pred = np.array(pred)
    
    # print table giving classifier accuracy using test_target
    print(classification_report(test_target, pred))

That is all, the developed Cosine-Similarity Model is very straight forward.

For the Scikit-Learn K-NN model, all we have to worry about is the value for the `n_neighbors` argument (number of neighbors to use for classification), the `weights` argument for `KNeighborsClassifier()` (how to vote), which we will just leave at its default value of `uniform`, as that is the same method used in the Cosine-Similarity Model.  Finally, we have the `algorithm` argument for `KNeighborsClassifier()`, which we will also leave at its default value of `auto`, as it will find the optimal algorithm to use for the given data. (Note, I ran a few tests with the `KNeighborsClassifier()` `metric` argument set to `cosine_similarity`.  I did not notice a difference so I will just be leaving `metric` at its default value).

In [12]:
def skl_knn(k, test_data, test_target, stored_data, stored_target):
    """k: number of neighbors to use in classication
    test_data: the data/targets used to test the classifier
    stored_data: the data/targets used to classify the test_data
    """
    
    classifier = KNeighborsClassifier(n_neighbors=k)  
    classifier.fit(stored_data, stored_target)

    y_pred = classifier.predict(test_data) 

    print(classification_report(test_target, y_pred))

### Testing
Now that we have the models setup, we will test how each model performs on each of the seven data sets.

For each data set/model pair, we will be measuring classification accuracy and speed of `test_data` classification.  For the Scikit-Learn model, a $k$ value of 5 will be used, and for the Cosine-Similarity model a $k$ value of 3 will be used, as those were the values found to be optimal (no techniques such as cross validation were used here for finding $k$, just performance on a few example tests).  

The tests on each data set alternate between the Cosine-Similarity Model (`cos_knn`) and the Scikit-Learn K-NN model (`skl_knn`).  

**NOTE**:  immidiately below I show the Cosine-Similarity Model run with a $k$ size of 5, to show that the major difference in classification speed between the Cosine-Similarity Model and the Scikit-Learn model does not have very much to do with the different sizes of $k$ (in the case below, using a larger $k$ value on the Cosine similarity model actually resulted in a faster classification time than when the model ran with a $k$ value of 3.  The classification time with a larger $k$ can vary, and is usually about 10-30 seconds slower than running with a $k$ value of 3).

In [13]:
%%time
cos_knn(5, test_img, test_target, sixty_x, sixty_y)

             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98      1025
        1.0       0.97      0.99      0.98      1111
        2.0       0.99      0.98      0.98       988
        3.0       0.98      0.97      0.97      1010
        4.0       0.99      0.97      0.98       976
        5.0       0.99      0.97      0.98       866
        6.0       0.99      0.99      0.99       948
        7.0       0.98      0.97      0.98      1034
        8.0       0.98      0.97      0.97       987
        9.0       0.95      0.97      0.96      1055

avg / total       0.98      0.98      0.98     10000

CPU times: user 5min 42s, sys: 1.37 s, total: 5min 43s
Wall time: 5min 20s


In [13]:
%%time
cos_knn(3, test_img, test_target, sixty_x, sixty_y)

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       958
        1.0       0.98      1.00      0.99      1105
        2.0       0.99      0.98      0.99      1032
        3.0       0.98      0.96      0.97      1021
        4.0       0.99      0.97      0.98      1015
        5.0       0.98      0.96      0.97       930
        6.0       0.98      0.99      0.99       958
        7.0       0.98      0.97      0.98       997
        8.0       0.97      0.97      0.97       958
        9.0       0.95      0.98      0.96      1026

avg / total       0.98      0.98      0.98     10000

CPU times: user 6min 1s, sys: 1.37 s, total: 6min 3s
Wall time: 5min 39s


In [14]:
%%time
skl_knn(5, test_img, test_target, sixty_x, sixty_y)

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       958
        1.0       0.96      1.00      0.98      1105
        2.0       0.99      0.97      0.98      1032
        3.0       0.97      0.97      0.97      1021
        4.0       0.98      0.97      0.98      1015
        5.0       0.96      0.96      0.96       930
        6.0       0.98      0.99      0.99       958
        7.0       0.96      0.97      0.97       997
        8.0       0.99      0.94      0.96       958
        9.0       0.95      0.97      0.96      1026

avg / total       0.97      0.97      0.97     10000

CPU times: user 9min 6s, sys: 676 ms, total: 9min 7s
Wall time: 9min 7s


In [15]:
%%time
cos_knn(3, test_img, test_target, fifty_x, fifty_y)

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99       958
        1.0       0.98      1.00      0.99      1105
        2.0       1.00      0.99      0.99      1032
        3.0       0.98      0.96      0.97      1021
        4.0       0.99      0.97      0.98      1015
        5.0       0.98      0.96      0.97       930
        6.0       0.98      0.99      0.99       958
        7.0       0.98      0.97      0.98       997
        8.0       0.97      0.97      0.97       958
        9.0       0.95      0.98      0.96      1026

avg / total       0.98      0.98      0.98     10000

CPU times: user 5min 5s, sys: 1.09 s, total: 5min 6s
Wall time: 4min 46s


In [16]:
%%time
skl_knn(5, test_img, test_target, fifty_x, fifty_y)

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99       958
        1.0       0.96      1.00      0.98      1105
        2.0       0.99      0.96      0.98      1032
        3.0       0.97      0.96      0.96      1021
        4.0       0.98      0.96      0.97      1015
        5.0       0.96      0.97      0.96       930
        6.0       0.98      0.99      0.99       958
        7.0       0.96      0.97      0.97       997
        8.0       0.99      0.93      0.95       958
        9.0       0.95      0.97      0.96      1026

avg / total       0.97      0.97      0.97     10000

CPU times: user 7min 57s, sys: 1.23 s, total: 7min 58s
Wall time: 7min 58s


In [17]:
%%time
cos_knn(3, test_img, test_target, fourty_x, fourty_y)

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99       958
        1.0       0.97      0.99      0.98      1105
        2.0       0.99      0.98      0.99      1032
        3.0       0.97      0.96      0.97      1021
        4.0       0.99      0.96      0.98      1015
        5.0       0.98      0.95      0.96       930
        6.0       0.98      0.99      0.98       958
        7.0       0.98      0.97      0.97       997
        8.0       0.96      0.97      0.96       958
        9.0       0.94      0.97      0.96      1026

avg / total       0.98      0.97      0.97     10000

CPU times: user 3min 53s, sys: 988 ms, total: 3min 54s
Wall time: 3min 38s


In [18]:
%%time
skl_knn(5, test_img, test_target, fourty_x, fourty_y)

             precision    recall  f1-score   support

        0.0       0.98      0.99      0.99       958
        1.0       0.95      1.00      0.97      1105
        2.0       0.99      0.96      0.97      1032
        3.0       0.96      0.96      0.96      1021
        4.0       0.98      0.96      0.97      1015
        5.0       0.96      0.96      0.96       930
        6.0       0.98      0.99      0.99       958
        7.0       0.96      0.97      0.97       997
        8.0       0.98      0.93      0.96       958
        9.0       0.95      0.97      0.96      1026

avg / total       0.97      0.97      0.97     10000

CPU times: user 6min 56s, sys: 96 ms, total: 6min 56s
Wall time: 6min 56s


In [19]:
%%time
cos_knn(3, test_img, test_target, thirty_x, thirty_y)

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99       958
        1.0       0.97      1.00      0.98      1105
        2.0       0.99      0.98      0.98      1032
        3.0       0.97      0.96      0.97      1021
        4.0       0.99      0.95      0.97      1015
        5.0       0.98      0.93      0.96       930
        6.0       0.97      0.99      0.98       958
        7.0       0.98      0.96      0.97       997
        8.0       0.96      0.96      0.96       958
        9.0       0.93      0.97      0.95      1026

avg / total       0.97      0.97      0.97     10000

CPU times: user 2min 55s, sys: 784 ms, total: 2min 56s
Wall time: 2min 44s


In [20]:
%%time
skl_knn(5, test_img, test_target, thirty_x, thirty_y)

             precision    recall  f1-score   support

        0.0       0.98      0.99      0.99       958
        1.0       0.94      1.00      0.97      1105
        2.0       0.98      0.95      0.96      1032
        3.0       0.96      0.96      0.96      1021
        4.0       0.98      0.95      0.96      1015
        5.0       0.95      0.95      0.95       930
        6.0       0.98      0.99      0.99       958
        7.0       0.95      0.97      0.96       997
        8.0       0.99      0.92      0.95       958
        9.0       0.94      0.97      0.95      1026

avg / total       0.96      0.96      0.96     10000

CPU times: user 4min 26s, sys: 640 ms, total: 4min 27s
Wall time: 4min 27s


In [21]:
%%time
cos_knn(3, test_img, test_target, twenty_x, twenty_y)

             precision    recall  f1-score   support

        0.0       0.97      1.00      0.98       958
        1.0       0.96      0.99      0.98      1105
        2.0       0.99      0.97      0.98      1032
        3.0       0.96      0.95      0.95      1021
        4.0       0.99      0.94      0.96      1015
        5.0       0.97      0.92      0.95       930
        6.0       0.97      0.99      0.98       958
        7.0       0.98      0.96      0.97       997
        8.0       0.95      0.94      0.95       958
        9.0       0.91      0.96      0.94      1026

avg / total       0.96      0.96      0.96     10000

CPU times: user 1min 59s, sys: 720 ms, total: 1min 59s
Wall time: 1min 51s


In [22]:
%%time
skl_knn(5, test_img, test_target, twenty_x, twenty_y)

             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98       958
        1.0       0.93      1.00      0.96      1105
        2.0       0.98      0.95      0.96      1032
        3.0       0.93      0.95      0.94      1021
        4.0       0.97      0.94      0.96      1015
        5.0       0.95      0.94      0.95       930
        6.0       0.97      0.99      0.98       958
        7.0       0.95      0.96      0.96       997
        8.0       0.99      0.90      0.94       958
        9.0       0.93      0.96      0.94      1026

avg / total       0.96      0.96      0.96     10000

CPU times: user 3min 23s, sys: 252 ms, total: 3min 24s
Wall time: 3min 24s


In [23]:
%%time
cos_knn(3, test_img, test_target, ten_x, ten_y)

             precision    recall  f1-score   support

        0.0       0.96      0.99      0.97       958
        1.0       0.96      0.99      0.97      1105
        2.0       0.99      0.96      0.97      1032
        3.0       0.95      0.94      0.94      1021
        4.0       0.98      0.92      0.95      1015
        5.0       0.96      0.91      0.94       930
        6.0       0.97      0.99      0.98       958
        7.0       0.97      0.96      0.97       997
        8.0       0.92      0.94      0.93       958
        9.0       0.91      0.95      0.93      1026

avg / total       0.96      0.96      0.96     10000

CPU times: user 59.7 s, sys: 360 ms, total: 1min
Wall time: 56 s


In [24]:
%%time
skl_knn(5, test_img, test_target, ten_x, ten_y)

             precision    recall  f1-score   support

        0.0       0.96      0.99      0.97       958
        1.0       0.91      1.00      0.95      1105
        2.0       0.97      0.93      0.95      1032
        3.0       0.93      0.94      0.93      1021
        4.0       0.97      0.93      0.95      1015
        5.0       0.94      0.93      0.94       930
        6.0       0.97      0.98      0.98       958
        7.0       0.94      0.94      0.94       997
        8.0       0.98      0.88      0.92       958
        9.0       0.91      0.94      0.93      1026

avg / total       0.95      0.95      0.95     10000

CPU times: user 1min 41s, sys: 312 ms, total: 1min 41s
Wall time: 1min 41s


In [25]:
%%time
cos_knn(3, test_img, test_target, one_x, one_y)

             precision    recall  f1-score   support

        0.0       0.85      0.98      0.91       958
        1.0       0.91      0.99      0.95      1105
        2.0       0.96      0.87      0.91      1032
        3.0       0.90      0.84      0.87      1021
        4.0       0.94      0.77      0.85      1015
        5.0       0.91      0.84      0.87       930
        6.0       0.95      0.94      0.94       958
        7.0       0.93      0.89      0.91       997
        8.0       0.82      0.88      0.85       958
        9.0       0.75      0.89      0.81      1026

avg / total       0.89      0.89      0.89     10000

CPU times: user 6.79 s, sys: 80 ms, total: 6.87 s
Wall time: 6.18 s


In [26]:
%%time
skl_knn(5, test_img, test_target, one_x, one_y)

             precision    recall  f1-score   support

        0.0       0.90      0.96      0.93       958
        1.0       0.75      1.00      0.86      1105
        2.0       0.97      0.80      0.88      1032
        3.0       0.87      0.87      0.87      1021
        4.0       0.87      0.79      0.83      1015
        5.0       0.86      0.85      0.85       930
        6.0       0.94      0.94      0.94       958
        7.0       0.84      0.88      0.86       997
        8.0       0.94      0.71      0.81       958
        9.0       0.78      0.82      0.80      1026

avg / total       0.87      0.86      0.86     10000

CPU times: user 10.8 s, sys: 20 ms, total: 10.8 s
Wall time: 10.8 s


The test results show the following:
* The Cosine-Similarity Model either matched the Scikit-Learn K-NN accuracy wise, or beat it by 1%-2%.  
* As far as classification speed goes, the Cosine-Similarity Model tends to be between 1.5-2 times faster than the Scikit-Learn K-NN.
* Strangely, the Cosine-Similarity Classifier tends to underperform when classifying the digit 9.  This could be on account of the fact that 4 and 9 are so similar, and thus tend to muddle the similarity metric for classification.  It is possible that the cosine similarity metric performs better on data with higher variance, where as other similarity metrics perform better on data with lower variance.

Below, the Cosine-Similarity Model and Scikit-Learn K-NN are tested in the time taken to classify a single image.

In [27]:
%%time
cos_knn(3, t_img, t_target, sixty_x, sixty_y)

             precision    recall  f1-score   support

        3.0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         1

CPU times: user 536 ms, sys: 236 ms, total: 772 ms
Wall time: 566 ms


In [28]:
%%time
skl_knn(5, t_img, t_target, sixty_x, sixty_y)

             precision    recall  f1-score   support

        3.0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         1

CPU times: user 25.5 s, sys: 200 ms, total: 25.7 s
Wall time: 25.3 s


As shown by the test results, the Cosine-Similarity Model is significantly faster (about 45 times faster in this example) at classifying single points than its corresponding Scikit-Learn classifier for larger stored data sets.


## Conclusion
Although I cannot speak as to why the Cosine-Similarity Model is generally more accurate than the Scikit-Learn K-NN model, it is likely faster because the Scikit-Learn model is far more complex.  The Scikit-Learn model takes many more arguments, adding complexity to the classification task, so it takes more time.  

The main lesson learned from this project is, depending on the problem being solved, it may be best to make an algorithm in-house instead of using pre-built algorithms, as these pre-built algorithms are meant to work well across a wide variety of situations, and are not optimized for just one problem.  


### Room For Improvement
Below are some points that can be used in the improvement of the Cosine-Similarity Model.

The current Cosine-Similarity Model uses a brute force approach to classifying input data, where each point to be classified has its distance computed with each other point in the model's stored data set.  This current brute force approach scales as $O(DN^2)$ for $D$ dimensions and $N$ samples.  To address this massive computational inefficiency, one of the many tree based data structures could be implemented into the model, greatly reducing the number of distance calculations made for each data point, furthur reducing classification complexity to $O[DN\text{log}(N)]$.

In the current Cosine-Similarity Model, each of the $k$ points used for voting are equally weighted.  While the optimal voting approach used for classification varies from problem to problem, it would be worth experimenting with other weighted voting methods.  Using cross validation to find the optimal $k$/weight "setting" for each problem would be worth the time.


### Lessons Learned & Moving Forward
Below is a list of the primary takeaways from the project, and possible future applications of the model.
* One of the greatest characteristics about K-NN is its simplicity.  The Cosine-Similarity Model, an incredibly simple model, out performs some standard convolutional neural networks accuracy wise.  This demonstrates the beauty of simple, parsimonious models, that seem to have been largely overlooked lately with the incredible performance of deep learning.
* Cosine similarity is often not used as it tends to violate the triangle inequality and does not work on negative data.  However, as this analysis proves, cosine similarity holds its own in certain problems, as it is generally both accurate and efficient.
* The results from this model are by no means state of the art, however they do show that proprietary algorithms often perform quite well on specific problems.
* I doubt the Cosine-Similarity Model will generalize well to more complex image classification tasks.  However, using techniques such as max pooling in more complex image classification tasks may help improve the model's accuracy.  
* In future projects, I would like to incorporate weighted voting, tree data structures, and cross-validation into the development of the model.


#### References
* [Scikit-Learn Documentation](http://scikit-learn.org/stable/modules/neighbors.html)